# Collect Darkchess Image or Video Datasets

In [ ]:
import os
import time

import cv2
import IPython.display as display
from datasets import Dataset
from datasets.features import Features, Image, ClassLabel

from config import CHESS
from eye.imgproc import (
    rotate,
    get_board_frame,
    get_form_frame,
    get_chess_frame
)

# ========================================
# Custom Settings
# ========================================
CAP_SRC = None  # set your IP camera URL, or video file path
SAVE_VIDEO_PATH = None  # set your video file path to save, or None to disable
SAVE_IMAGE_DIR = None # set your image directory to save, or None to disable
DISPLAY_CAP = False # display captured frame

SPECIFIC_CHESS_LABEL = None
CHESS_IMAGE_NUM = 144000 # 50 x 360 x 8 = 144000
image_data = {
    "image": [],
    "label": []
}

# Example: process board frame to chess frame and save
def process_image_to_save(frame, frame_count) -> None:
    if frame_count % 5 != 0 or len(image_data["image"]) >= CHESS_IMAGE_NUM:
        return

    for angle in range(360):
        for shift in range(8):
            img = get_board_frame(frame)
            img = get_form_frame(img, 20)
            img = get_chess_frame(img, shift)
            img = rotate(img, angle)
            image_data["image"].append(img)
            image_data["label"].append(SPECIFIC_CHESS_LABEL)

    print(f"Now image data number: {len(image_data['image'])}", end="\r")

# ========================================
# End of Custom Settings
# ========================================

cap = cv2.VideoCapture(CAP_SRC)
display_handle = display.display(None, display_id=True)
out = cv2.VideoWriter(
    SAVE_VIDEO_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"), 
    10,
    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)

while True:
    try:
        frame = cap.read()[1]
        if frame is None:
            print("\nEnd of video")
            break

        if SAVE_IMAGE_DIR:
            os.makedirs(SAVE_IMAGE_DIR, exist_ok=True)
            process_image_to_save(frame, int(cap.get(cv2.CAP_PROP_POS_FRAMES)))

        if SAVE_VIDEO_PATH:
            os.makedirs(os.path.dirname(SAVE_VIDEO_PATH), exist_ok=True)
            out.write(frame)
        
        if DISPLAY_CAP:
            time.sleep(0.5)  # 2 fps, buffer time
            encoded_frame = cv2.imencode(".jpg", frame)[1]
            display_handle.update(display.Image(data=encoded_frame.tobytes()))

    except KeyboardInterrupt:
        break

cap.release()
out.release()
display_handle.update(None)
if SAVE_IMAGE_DIR:
    print(f"Saving {len(image_data['image'])} images to {SAVE_IMAGE_DIR}")
    image_datasets = Dataset.from_dict(
        mapping=image_data,
        features=Features({"image": Image("L"), "label": ClassLabel(num_classes=len(CHESS[:-2]), names=[item["code"] for item in CHESS[:-2]])})
    ).train_test_split(train_size=0.8, test_size=0.2, shuffle=False)
    image_datasets.save_to_disk(SAVE_IMAGE_DIR)

# Login to HuggingFace (just login once)

In [ ]:
from huggingface_hub import interpreter_login
interpreter_login()

# Push Board Dataset to Hugging Face
> manually write `metadata.jsonl` is required.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("datasets/darkchess-board")
dataset.push_to_hub("ryanlinjui/darkchess-board")

# Combine All Chess Datasets and Push it

In [ ]:
from datasets import DatasetDict
from datasets import load_from_disk, concatenate_datasets

from config import CHESS

train_datasets = []
test_datasets = []

for i in range(len(CHESS[:-2])):
    print(f"Loading dataset {i}", end="\r")
    dataset = load_from_disk(f"datasets/darkchess-single/{i}")
    train_datasets.append(dataset["train"])
    test_datasets.append(dataset["test"])

datasets = DatasetDict({
    "train": concatenate_datasets(train_datasets),
    "test": concatenate_datasets(test_datasets)
})
datasets.push_to_hub("ryanlinjui/darkchess-single")

# Load Dataset from HuggingFace

In [ ]:
import numpy as np
from datasets import load_dataset
from tensorflow.keras.utils import img_to_array

datasets = load_dataset("ryanlinjui/darkchess-single")

train_images = np.array([img_to_array(image["image"]) for image in datasets["train"]]).astype("float32") / 255.0
train_labels = np.array([image["label"] for image in datasets["train"]])

test_images = np.array([img_to_array(image["image"]) for image in datasets["test"]]).astype("float32") / 255.0
test_labels = np.array([image["label"] for image in datasets["test"]])

# List devices, check for CPU, GPU, MPU availability

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

# Construct the model

In [4]:
from keras import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), input_shape=(56, 56, 1), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 2), strides=(1, 1), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=4096, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=4096, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1000, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=14, activation="softmax"))

model.compile(
    optimizer=Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
    loss = "categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model

In [ ]:
gray_frame1 = np.reshape(gray_frame,[-1, 56, 56, 1])
gray_label1 = np_utils.to_categorical(gray_label, num_classes=14)
history = model.fit(x=gray_frame1, y=gray_label1, epochs=15, batch_size=360)

# Evaluate the model

In [ ]:
history = model.evaluate(x=test_images, y=test_labels, batch_size=360)

# Check predictions on test data

In [ ]:
plt.figure(figsize=(10,10))
predict_index = 0
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    a = gray_frame[predict_index+i].reshape([-1,56,56,1])
    m = model.predict(a)
    plt.imshow(gray_frame[predict_index+i])
    plt.xlabel(m.argmax())
plt.show()

In [ ]:
model.save("cnn_darkchess_model.h5")
model.save_weights("cnn_darkchess_weights.h5")

# Push Model to Hugging Face

In [ ]:
import os

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from huggingface_hub import push_to_hub_keras

from config import DEFAULT_MODEL_NAME, DEFAULT_WEIGHTS_NAME

model_dir = ""

model = load_model(os.path.join(model_dir, DEFAULT_MODEL_NAME))
model.load_weights(os.path.join(model_dir, DEFAULT_WEIGHTS_NAME))
plot_model(model, show_shapes=True, show_dtype=True, show_layer_activations=True, expand_nested=True)
model.summary()

push_to_hub_keras(model, "")

In [ ]:
import pandas as pd 
predictions = model.predict_classes(gray_frame1) 
pd.crosstab(gray_label, predictions, rownames=["實際值"], colnames=["預測值"])

In [ ]:
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["loss"], label = "loss")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([-0.05, 1.05])
plt.xlim([0, 5])
plt.legend(loc="lower right")